# Example

A stock market example of a schema mismatch:

## TSE:

stock | date | type | price
------|------|------|-------
IBM   | -    | open | 50
XON   | -    | close| 60
MSFT  | -    | high | 70
GOOG  | -    | low  | 40

## NYSE:

IBM:

data | type | price
-----|------|------
-    | -    | -

MSFT:

data | type | price
-----|------|------
-    | -    | -

## HKSE
stock | date | open | close | high | low
------|------|------|-------|------|-----
-     | -    | -    | -     | -    | -

# Challenges
Say we wanted to compare stock prices across multiple stock exchanges. Joining across these tables would be problematic:
* Between TSE and NYSE, we need to query a **stock** tuple value against a **stock** table name!
* Between TSE and HKSE, we have to compare the value inside **type** against a whole attribute **open/close/high/low**

# Data Cubes

## Example: Marginals
* 4-dimentional **Sales** cube includes the sum of *price* over each bar, beer, drinker, and time unit (eg. days)
* It would also include sum of *price* over all bar-beer pairs, all bar-drinker-day typles **each of which are a *group-by* query with some aggregate functions**
* Similar comments apply to the *gratuity* data

In general, a cube contains multiple measures
* Data warehouses may contain multiple cubes
* Each cube has facts and dimensions
* Can be organized in tables (*ROLAP*) or multidimensional storage (*MOLAP*)
    * *ROLAP* can use star or snowflake schema

## Structure of the Cube
Dimensions have different members (eg. bars, drinkers, etc)

Members are organized in a hierarchy
* bars into cities, cities into states, etc.
* beers into manufacturers, manufacturers into countries
Some dimensions may permit multiple hierarchies:
* days into months, months into quarters; days also into weeks, etc.
These hierarchies alow us to visualize data at multiple granularities
* eg. the sales of all Czech beers in Joe's bar during Q2 of 2014

### Aggregation
**Drill-down** is to *de-aggregate* data  
**Roll-up** is to *aggregate* data along one or more dimensions

#### Example

Bar        | Jim | Bob | Mary
-----------|-----|-----|-----
Joe's Bar  | 45  | 33  | 30
Nut House  | 50  | 36  | 42
Blue Chalk | 38  | 31  | 40

**Roll up by Bar:**

Jim | Bob | Mary
----|-----|-----
133 | 100 | 112

**Drill down by Beer**:


Beer     | Jim | Bob | Mary
---------|-----|-----|-----
Bud      | 40  | 29  | 40
M'lob    | 45  | 31  | 37
Bud Lite | 48  | 40  | 35

Notice how in order to drill down, you need access to some additional data that's probably stored in another cube somewhere.

#### Example
**Measure**: Sales  
**Dimensions**
* Products (key: *pid*)
* Locations (*locid*)
* Times (*timeid*)



# Fact and Dimension Tables

#### Example
Fact tables (can be combined into one)
* `Sales(*pid, *timeid, *locid, sales)`
* `Profits(*pid, *timeid, *locid, profit)`
* `Costs(*pid, *timeid, *locid, cost)`

Dimension Tables
* `Locations(*locid, city, state, country)`
* `Products(*pid, pname, category, price)`
* `Times(*timeid, date, week, month, year, holiday_flag)`

## Date/Time Dimension
Date/Time is a special kind of dimension
* eg. fixed attribute values (November always has 30 days, leap years are known, etc)

Some useful OLAP functions:
* eg. durations/timespans, fiscal years, calendar years

Used in **role-playing**
* eg. used in OrderDate, ShippingDate, DueDate

## Modelling of Dimensions
Two main models:

### Star Schema: table per dimension
* simplicity 
* easier to formulate queries (one join per dimension, AKA **star join**)
* poor modelling capabilities: what if dimension hiearchy is unbalanced or heterogenous?

### Snowflake schema: table per level of hierarchy per dimension
* more flexiblity than star, more encapsulation of data in each table
* heterogenous dimension hierarchies still problematic
* query formations more complicated, more joins/dimension

#### Example

```
Time(timeId, dayId, monthId, date).
Time1(dayId, day-of-week).
Time2(monthId, yearId, month).
Time3(yearId, year, is-leap-year).
```

We can have more specific information in each subtable. We lose on performance though.